In [1]:
# Skeletonization image

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image 
import PIL

In [8]:
cam = cv2.VideoCapture(0)

cv2.namedWindow("Take iamge")

img_counter = 0

while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    cv2.imshow("test", frame)

    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed to leave
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed to take picture
        img_name = "opencv_frame_{}.png".format(img_counter)
        cv2.imwrite(img_name, frame)
        print("{} written!".format(img_name))
        img_counter += 1

cam.release()

cv2.destroyAllWindows()

opencv_frame_0.png written!
Escape hit, closing...


In [13]:
# Read the image as a grayscale image
# img = cv2.imread('test.jpg', 0)
# ret,img = cv2.threshold(img, 127, 255, 0)
# size = np.size(img)
# skel = np.zeros(img.shape, np.uint8)
# element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
# while True:
#     open = cv2.morphologyEx(img, cv2.MORPH_OPEN, element)
#     temp = cv2.subtract(img, open)
#     eroded = cv2.erode(img, element)
#     skel = cv2.bitwise_or(skel,temp)
#     img = eroded.copy()
#     if cv2.countNonZero(img)==0:
#         break
# cv2.imwrite("test1.jpg",skel)
# cv2.imshow("Skeleton",skel)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# print("Quit success")


In [3]:
import torch

class HandGestureNet(torch.nn.Module):
    """
    citation:
    ------------
        @inproceedings{devineau2018deep,
            title={Deep learning for hand gesture recognition on skeletal data},
            author={Devineau, Guillaume and Moutarde, Fabien and Xi, Wang and Yang, Jie},
            booktitle={2018 13th IEEE International Conference on Automatic Face \& Gesture Recognition (FG 2018)},
            pages={106--113},
            year={2018},
            organization={IEEE}
        }
    """
    
    def __init__(self, n_channels=66, n_classes=14, dropout_probability=0.2):

        super(HandGestureNet, self).__init__()
        
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.dropout_probability = dropout_probability
        self.all_conv_high = torch.nn.ModuleList([torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=1, out_channels=8, kernel_size=7, padding=3),
            torch.nn.ReLU(),
            torch.nn.AvgPool1d(2),

            torch.nn.Conv1d(in_channels=8, out_channels=4, kernel_size=7, padding=3),
            torch.nn.ReLU(),
            torch.nn.AvgPool1d(2),

            torch.nn.Conv1d(in_channels=4, out_channels=4, kernel_size=7, padding=3),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=self.dropout_probability),
            torch.nn.AvgPool1d(2)
        ) for joint in range(n_channels)])

        self.all_conv_low = torch.nn.ModuleList([torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=1, out_channels=8, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.AvgPool1d(2),

            torch.nn.Conv1d(in_channels=8, out_channels=4, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.AvgPool1d(2),

            torch.nn.Conv1d(in_channels=4, out_channels=4, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=self.dropout_probability),
            torch.nn.AvgPool1d(2)
        ) for joint in range(n_channels)])

        self.all_residual = torch.nn.ModuleList([torch.nn.Sequential(
            torch.nn.AvgPool1d(2),
            torch.nn.AvgPool1d(2),
            torch.nn.AvgPool1d(2)
        ) for joint in range(n_channels)])

        self.fc = torch.nn.Sequential(
            torch.nn.Linear(in_features=9 * n_channels * 12, out_features=1936),  # <-- 12: depends of the sequences lengths (cf. below)
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=1936, out_features=n_classes)
        )

        for module in itertools.chain(self.all_conv_high, self.all_conv_low, self.all_residual):
            for layer in module:
                if layer.__class__.__name__ == "Conv1d":
                    torch.nn.init.xavier_uniform_(layer.weight, gain=torch.nn.init.calculate_gain('relu'))
                    torch.nn.init.constant_(layer.bias, 0.1)

        for layer in self.fc:
            if layer.__class__.__name__ == "Linear":
                torch.nn.init.xavier_uniform_(layer.weight, gain=torch.nn.init.calculate_gain('relu'))
                torch.nn.init.constant_(layer.bias, 0.1)

    def forward(self, input):
        all_features = []

        for channel in range(0, self.n_channels):
            input_channel = input[:, :, channel]
            input_channel = input_channel.unsqueeze(1)

            high = self.all_conv_high[channel](input_channel)
            low = self.all_conv_low[channel](input_channel)
            ap_residual = self.all_residual[channel](input_channel)
            output_channel = torch.cat([
                high,
                low,
                ap_residual
            ], dim=1)
            all_features.append(output_channel)

        all_features = torch.cat(all_features, dim=1)

        all_features = all_features.view(-1, 9 * self.n_channels * 12) 
        output = self.fc(all_features)

        return output

In [4]:
import itertools
model = HandGestureNet(n_channels=66, n_classes=14)
model.load_state_dict(torch.load('gesture_pretrained_model.pt'))

<All keys matched successfully>

In [9]:
from PIL import Image
import torchvision.transforms as transforms
# Dictionary of hand gesture
dict = {
    0: "Grab",
    1: "Tap",
    2:"Expand",
    3:"Pinch",
    4:"Rotation Clockwis",
    5:"Rotation Counter Clockwise",
    6:"Swipe Right",
    7:"Swipe Left",
    8:"Swipe Up",
    9:"Swipe Down",
    10:"Swipe X",
    11:"Swipe +",
    12:"Swipe V",
    13:"Shake"
}
# Read the image
image = Image.open('opencv_frame_0.png')

# Define a transform to convert the image to tensor
transform = transforms.ToTensor()

# Convert the image to PyTorch tensor
tensor = transform(image)
predictions = model(tensor)
_, predictions = predictions.max(dim=1)
ll = predictions.tolist()
for k in ll:
    print("detected :", dict[k])

print( "detected as:", dict[max(set(ll), key=ll.count)])

detected : Tap
detected : Rotation Counter Clockwise
detected : Shake
detected : Tap
detected : Shake
detected : Tap
detected : Rotation Counter Clockwise
detected : Shake
detected : Shake
detected : Swipe Left
detected : Tap
detected : Rotation Counter Clockwise
detected : Shake
detected : Swipe Right
detected : Shake
detected as: Shake


In [5]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import load_model

In [35]:
# initialize mediapipe
# mpHands = mp.solutions.hands
# hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
# mpDraw = mp.solutions.drawing_utils

In [14]:
from PIL import Image
import torchvision.transforms as transforms
cam = cv2.VideoCapture(0)
dict = {
    0: "Grab",
    1: "Tap",
    2:"Expand",
    3:"Pinch",
    4:"Rotation Clockwis",
    5:"Rotation Counter Clockwise",
    6:"Swipe Right",
    7:"Swipe Left",
    8:"Swipe Up",
    9:"Swipe Down",
    10:"Swipe X",
    11:"Swipe +",
    12:"Swipe V",
    13:"Shake"
}
while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    cv2.imshow("test", frame)

    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed to leave
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed to take picture
        img_name = "opencv_frame_{}.png".format(img_counter)
        cv2.imwrite(img_name, frame)
        print("{} written!".format(img_name))
        img_counter += 1
    x , y, c = frame.shape
  # Flip the frame vertically
    frame = cv2.flip(frame, 1)
    transform = transforms.ToTensor()
    # Convert the image to PyTorch tensor
    tensor = transform(frame)
    predictions = model(tensor)
    _, predictions = predictions.max(dim=1)
    ll = predictions.tolist()
#     print( "detected as:", dict[max(set(ll), key=ll.count)])
    font = cv2.FONT_HERSHEY_SIMPLEX
    result = dict[max(set(ll), key=ll.count)]
    # Use putText() method for
    # inserting text on video
    names = [dict[k] for k in ll]
    names = ' '.join(names)
    print("all detection results: ", names)
    cv2.putText(frame, 
                result, 
                (50, 50), 
                font, 1, 
                (0, 255, 255), 
                2, 
                cv2.LINE_4)
    # Display the resulting frame
    cv2.imshow('video', frame)
cam.release()

cv2.destroyAllWindows()

all detection results:  Tap Rotation Counter Clockwise Shake Tap Swipe Left Tap Rotation Clockwis Shake Shake Shake Tap Rotation Clockwis Shake Shake Shake
all detection results:  Tap Rotation Clockwis Shake Tap Shake Swipe Left Rotation Clockwis Shake Shake Shake Tap Rotation Counter Clockwise Shake Shake Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Shake Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Clockwis Shake Tap Swipe Left Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Clockwis Shake Shake Shake
all detection results:  Tap Rotation Clockwis Shake Tap Shake Tap Rotation Clockwis Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwis

all detection results:  Tap Rotation Counter Clockwise Shake Shake Swipe Left Tap Rotation Clockwis Shake Tap Shake Tap Rotation Clockwis Shake Shake Shake
all detection results:  Tap Rotation Counter Clockwise Shake Shake Shake Tap Rotation Clockwis Shake Tap Swipe Left Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Shake Shake Tap Rotation Clockwis Shake Tap Swipe Left Shake Rotation Clockwis Shake Tap Swipe Left
all detection results:  Tap Rotation Counter Clockwise Shake Shake Swipe Left Tap Rotation Counter Clockwise Shake Tap Swipe Left Tap Rotation Clockwis Shake Shake Swipe Left
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Clockwis Shake Shake Swipe Left
all detection results:  Tap Rotation Counter Clockwise Shake Shake Shake Tap Rotation Clockwis Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  

all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Clockwis Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Swipe Right Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Swipe Left Tap Rotation Clockwis Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Clockwis Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Clockwis Shake Tap Swipe Left Tap Rotation Clockwis Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Co

all detection results:  Tap Rotation Counter Clockwise Shake Shake Swipe Left Tap Rotation Clockwis Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Swipe Left Tap Rotation Counter Clockwise Shake Tap Swipe Left Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Swipe Left
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Clockwis Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Shake Shake Tap Rotation Clockwis Shake Shake Swipe Left Tap Rotation Clockwis Shake Tap Shake
all detection r

all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Shake Rotation Counter Clockwise Shake Tap Swipe Left Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Shake Shake
all detection results:  Tap Rotation Counter Clockwise Shake Shake Shake Shake Rotation Clockwis Shake Tap Shake Tap Rotation Clockwis Shake Tap Shake
all detection results:  Tap Rotation Clockwis Shake Tap Swipe Right Tap Rotation Counter Clockwise Shake Tap Swipe Left Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Swipe Left Tap Rotation Counter Clockwise Shake Tap Swipe Left
all

...

all detection results:  Tap Rotation Counter Clockwise Shake Tap Swipe Left Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Clockwis Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Swipe Left Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Clockwis Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Clockwis Shake Tap Swipe Left Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:

all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Clockwis Shake Tap Shake Tap Rotation Clockwis Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap 

all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Clockwis Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Clockwis Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Clockwis Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation 

all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Clockwis Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection resul

all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Clockwis Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Clockwis Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake Tap Rotation Counter Clockwise Shake Tap Shake
all detection results:  Tap 